# 模型参数的访问、初始化和共享
本节利用`nn`中的`init`模块对参数进行初始化，`init`模块包含多种模型初始化方法

我们先定义一个与上一节中相同的含单隐藏层的多层感知机。我们依然使用默认方式初始化它的参数，并做一次前向计算。与之前不同的是，在这里我们从`nn`中导入了`init`模块

In [1]:
import torch as t
from torch import nn
from torch.nn import init

In [2]:
net=nn.Sequential(nn.Linear(4,3),nn.ReLU(),nn.Linear(3,1))
print(net)
X=t.rand(2,4)
Y=net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


## 访问模型的参数
对于`Sequential`实例中含模型参数的层，我们可以通过`Module`类的`parameters()`或者`named_parameters`方法来访问所有参数（以迭代器的形式返回），后者除了返回参数Tensor外还会返回其名字

In [3]:
print('类型为generator的class：',type(net.named_parameters()))
for name,param in net.named_parameters():
    print(name,param.size())
#print('\n')
#for name,param in net.parameters():会报错，因为parameters方法没有name属性
#    print(name,param.size())

类型为generator的class： <class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


可见返回的名字自动加上了层数的索引作为前缀：0层权重，0层偏置。

我们再来访问`net`中单层的参数。对于使用`Sequential`类构造的神经网络，我们可以通过<font color=red>方括号\[ \]</font>来访问网络的任一层。**索引0表示隐藏层为Sequential实例最先添加的层。**

In [4]:
for name,param in net[0].named_parameters():
    print(name,param.size(),type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


返回的`param`的类型为`torch.nn.parameter.Parameter`，其实这是`Tensor`的子类，和Tensor不同的是如果一个Tensor是Parameter，那么它会自动被添加到模型的参数列表里，如下例所示：

In [5]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(t.rand(20, 20))
        self.weight2 = t.rand(20, 20)
    def forward(self, x):
        pass

n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


上面的代码中weight1在参数列表中但是weight2却没在参数列表中。

因为Parameter是Tensor，即Tensor拥有的属性它都有，比如可以根据data来访问参数数值，用grad来访问参数梯度。

In [6]:
weight_0=list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad) # 反向传播前梯度为None
Y.backward()
print(weight_0.grad)

tensor([[-0.2172,  0.2207, -0.3987,  0.1376],
        [-0.4360,  0.3091,  0.1757, -0.3798],
        [ 0.0065,  0.0448,  0.0394,  0.3033]])
None
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.2249, -0.1948, -0.2118, -0.1635]])


## 初始化模型参数
PyTorch的`init`模块里提供了多种预设的初始化方法。在下面的例子中，我们将权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并依然将偏差参数清零

In [7]:
for name,param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param,mean=0,std=0.01)   # init.normal_(tensor, mean=0.0, std=1.0) 
        # 第一个参数是Tensor
        print(name,param.data)

0.weight tensor([[ 0.0078,  0.0116, -0.0016, -0.0123],
        [ 0.0020, -0.0049,  0.0059,  0.0163],
        [ 0.0162,  0.0060,  0.0008,  0.0003]])
2.weight tensor([[-0.0050,  0.0044,  0.0144]])


In [10]:
# 初始化偏置
for name,param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param,val=0)# Fills the input Tensor with the value :math:`\text{val}`.
        print(name,param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


## 自定义初始化方法
有时候我们需要的初始化方法并没有在`init`模块中提供。这时，可以实现一个初始化方法，从而能够像使用其他初始化方法那样使用它。

`init`模块里面内嵌的初始化方法都是用inlpace方法改变Tensor值得函数，因而该过程是不记录梯度的。因而我们也可以据此自定义初始化方法(如下例)：

In [13]:
def init_weight_(tensor):
    with t.no_grad():# 还记得no_grad()吗，防止跟踪历史记录
        tensor.uniform_(-10,10)
        tensor*=(tensor.abs()>=5).float()
        
for name,para in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name,param.data)

0.weight tensor([[ 7.7220,  0.0000,  5.8931, -0.0000],
        [ 9.1177,  9.9543, -0.0000, -0.0000],
        [-0.0000, -7.0933, -0.0000, -0.0000]])
2.weight tensor([[-0.0000,  0.0000, -0.0000,  6.0056],
        [ 5.9666,  8.1709,  0.0000,  0.0000],
        [-0.0000,  7.8371, -9.3119, -8.3508]])


## 共享模型参数
在有些情况下，我们希望在多个层之间共享模型参数。之前提到了如何共享模型参数: Module类的forward函数里多次调用同一个层。此外，如果我们传入Sequential的模块是同一个Module实例的话参数也是共享的，如下例：

In [15]:
linear=nn.Linear(1,1,bias=False)
net=nn.Sequential(linear,linear)
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [17]:
# 在内存中，这两个线性层其实是一个对象:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


因为模型参数里包含了梯度，所以在反向传播计算时，这些共享的参数的梯度是累加的:

In [18]:
x = t.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad) # 单次梯度是3，两次所以就是6

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
